In [1]:
import mxnet as mx
import numpy as np

/home/users/benjin.zhu/.libs/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def get_symbol(num_classes, dtype='float32', **kwargs):
    input_data = mx.sym.Variable(name='data')
    if dtype == 'float16':
        input_data = mx.sym.Cast(input_data, dtype=np.float16)
    # Stage 1
    conv1 = mx.sym.Convolution(name='conv1', data=input_data, kernel=(11, 11), stride=(4, 4), num_filter=96)
    relu1 = mx.sym.Activation(data=conv1, act_type='relu')
    lrn1 = mx.sym.LRN(data=relu1, alpha=0.0001, beta=0.75, knorm=2, nsize=5)
    pool1 = mx.sym.Pooling(data=lrn1, pool_type='max', kernel=(3, 3), stride=(2, 2))
    # Stage 2
    conv2 = mx.sym.Convolution(name='conv2', data=pool1, kernel=(5, 5), pad=(3, 3), num_filter=256)
    relu2 = mx.sym.Activation(data=conv2, act_type='relu')
    lrn2 = mx.sym.LRN(data=relu2, alpha=0.0001, beta=0.75, knorm=2, nsize=5)
    pool2 = mx.sym.Pooling(data=lrn2, pool_type='max', kernel=(3, 3), stride=(2, 2))
    # Stage 3
    conv3 = mx.sym.Convolution(name='conv3', data=pool2, kernel=(3, 3), pad=(1, 1), num_filter=384)
    relu3 = mx.sym.Activation(data=conv3, act_type="relu")
    conv4 = mx.sym.Convolution(name='conv4', data=relu3, kernel=(3, 3), pad=(1, 1), num_filter=384)
    relu4 = mx.sym.Activation(data=conv4, act_type="relu")
    conv5 = mx.sym.Convolution(name='conv5', data=relu4, kernel=(3, 3), pad=(1, 1), num_filter=256)
    relu5 = mx.sym.Activation(data=conv5, act_type="relu")
    pool3 = mx.sym.Pooling(data=relu5, kernel=(3, 3), stride=(2, 2), pool_type='max')
    # Stage 4
    flatten = mx.sym.Flatten(pool3)
    fc1 = mx.sym.FullyConnected(data=flatten, name='fc1', num_hidden=4096)
    relu6 = mx.sym.Activation(data=fc1, act_type='relu')
    dropout1 = mx.sym.Dropout(data=relu6, p=0.5)
    # Stage 5
    fc2 = mx.sym.FullyConnected(name='fc2', data=dropout1, num_hidden=4096)
    relu7 = mx.sym.Activation(data=fc2, act_type="relu")
    dropout2 = mx.sym.Dropout(data=relu7, p=0.5)
    # Stage 6
    fc3 = mx.sym.FullyConnected(data=dropout2, name='fc3', num_hidden=num_classes)
    if dtype == 'float16':
        fc3 = mx.sym.Cast(fc3, dtype=np.float32)
    smo = mx.sym.SoftmaxOutput(data=fc3, name='softmax')
    
    return smo